In [1]:
import os

import torch
import torch.nn as nn
import transformers
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from datasets import load_dataset

from peft import AdaptionPromptConfig, get_peft_model, TaskType, prepare_model_for_kbit_training
from utils.prompter import Prompter
from utils.make_dataset import DatasetCreator

os.environ["MASTER_ADDR"] = "localhost"
os.environ["MASTER_PORT"] = "9994"  # modify if RuntimeError: Address already in use
os.environ["RANK"] = "0"
os.environ["LOCAL_RANK"] = "0"
os.environ["WORLD_SIZE"] = "1"

/data/chris/anaconda3/envs/peft-env/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA SETUP: CUDA runtime path found: /data/chris/anaconda3/envs/peft-env/lib/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 6.1
CUDA SETUP: Detected CUDA version 116
CUDA SETUP: Loading binary /data/chris/anaconda3/envs/peft-env/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cuda116_nocublaslt.so...


/data/chris/anaconda3/envs/peft-env/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: Compute capability < 7.5 detected! Only slow 8-bit matmul is supported for your GPU!
  warn(msg)


In [14]:
# Hyperparameters
model_name = "huggyllama/llama-7b"
# model_name = "facebook/opt-1.3b"
adapter_len = 10 
adapter_layers = 30
cutoff_len = 256
train_on_inputs = True
add_eos_token = False
batch_size: int = 1
micro_batch_size: int = 1
num_epochs: int = 3
learning_rate: float = 3e-4
cutoff_len: int = 256
val_set_size: int = 2000
gradient_accumulation_steps = 1

In [15]:
model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=model_name,
    load_in_8bit=True,
    torch_dtype=torch.float16,
    device_map="auto"
)

tokenizer = AutoTokenizer.from_pretrained(model_name)

Loading checkpoint shards: 100%|██████████| 2/2 [00:11<00:00,  5.64s/it]


In [16]:
model = prepare_model_for_kbit_training(model)
peft_config = AdaptionPromptConfig(adapter_layers=1, adapter_len=4, task_type="CAUSAL_LM")
model = get_peft_model(model, peft_config)

In [17]:
model.print_trainable_parameters()

trainable params: 16385 || all params: 6738432001 || trainable%: 0.00024315745855368765


In [6]:
data_path = "alpaca_data_gpt4.json"
data = load_dataset("json", data_files=data_path)

val_set_size = 0.1
dataset_creator = DatasetCreator(
    tokenizer=tokenizer,
    prompter=Prompter(template_name="alpaca"),
    cutoff_len=cutoff_len,
    train_on_inputs=train_on_inputs,
    add_eos_token=add_eos_token,
    val_set_size=val_set_size,
)

dataset = dataset_creator.create_dataset(data)
train_set = dataset["train"]
val_set = dataset["val"]

Using custom data configuration default-e02b8a396fbb2bb4
Found cached dataset json (/home/eecs/christopherchou/.cache/huggingface/datasets/json/default-e02b8a396fbb2bb4/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51)
100%|██████████| 1/1 [00:00<00:00, 202.62it/s]
Loading cached split indices for dataset at /home/eecs/christopherchou/.cache/huggingface/datasets/json/default-e02b8a396fbb2bb4/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51/cache-c4a703fef6e6f08c.arrow and /home/eecs/christopherchou/.cache/huggingface/datasets/json/default-e02b8a396fbb2bb4/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51/cache-fcabdebf1e613dd6.arrow
100%|██████████| 5201/5201 [00:08<00:00, 611.54ex/s]


In [7]:
model.config.use_cache = False

training_args = transformers.TrainingArguments(
    output_dir="./llama-adapter-7b",
    learning_rate=learning_rate,
    overwrite_output_dir=True,
    num_train_epochs=num_epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    fp16=True,
    optim="adamw_torch",
    evaluation_strategy="epoch",
    logging_steps=1,
    save_steps=3,
    deepspeed="ds_config.json",
    gradient_checkpointing=True,
)

trainer = transformers.Trainer(
    model=model,
    args=training_args,
    train_dataset=train_set,
    eval_dataset=val_set,
    tokenizer=tokenizer,
)

In [9]:
trainer.train()

OutOfMemoryError: CUDA out of memory. Tried to allocate 86.00 MiB (GPU 0; 11.90 GiB total capacity; 11.01 GiB already allocated; 71.69 MiB free; 11.18 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF